In [1]:
using CSV, Dates, JuliaDB

In [2]:
ionex_file = "../jplg3660.12i"

"../jplg3660.12i"

In [3]:
# OPen the file and place its contents in an array name lines
file = open(ionex_file);
lines = readlines(file)
close(file)

In [4]:
using Dates
index = 0
tec_values = []   # Tec values per longitud, lat(-180,180)
tec = []          # All the tec values in a file
dates_array = []  # Store the different time stamps

for line in lines
    index += 1
    
    # Obtain time_stamp and tec values for every map (13 TEC maps in a file)
    if occursin(r"START OF TEC MAP", line)
        date_line = lines[index+1]
        date_units = []
        string = split(date_line)
        
        for x in 1:6
            push!(date_units,parse(Int,string[x]))
        end
        
        # Date units 
        year = date_units[1]
        month = date_units[2]
        day = date_units[3]
        hour = date_units[4]
        minutes = date_units[5]
        seconds = date_units[6]
        
        # Map time_stamp for a period(2 hours)
        time_stamp = DateTime(year, month, day, hour, minutes, seconds)
        push!(dates_array, time_stamp)
    end
    
    # Tec values per latitud in the longitud range (-180,180)
    if occursin(r"LAT/LON1/LON2/DLON/H", line)
        for x in 1:5
            tec_values = split(lines[index + x])
            for n in tec_values
                push!(tec, n)
            end
        end
    end
    
end

In [5]:
# used to create a latitud, to hold 2-hour-interval Tec map data
function create_coordinates(start, stop, step)
    coordinates = []
    for i in range(start, stop=stop, step=step)
        for j in 1:73
            push!(coordinates,i)
        end
    end
    coordinates # Julia automatically returns the last statement
end
    

create_coordinates (generic function with 1 method)

In [6]:
latitud = create_coordinates(87.5, -87.5, -2.5)
longitud = collect(Iterators.flatten([collect(-180:5:180) for i in 1:71])) # I Still don't grok this!

5183-element Array{Int64,1}:
 -180
 -175
 -170
 -165
 -160
 -155
 -150
 -145
 -140
 -135
 -130
 -125
 -120
    ⋮
  125
  130
  135
  140
  145
  150
  155
  160
  165
  170
  175
  180

In [18]:
# Create CSV for daily tec map data.
start = 1
map_tec_values = 5183  # Total tec values in a 2 hour map period
stop = map_tec_values
for d in dates_array
    print("\nStart ", start)
    #print("\nStop ", stop)
    time_stamp = [d for i in 1:map_tec_values]
    tec_value = [parse(Int,value) for value in tec[start:stop]]
    t = table((time_stamp=time_stamp, latitud=latitud, longitud=longitud, tec_value=tec_value); pkey = [:time_stamp, :latitud, :longitud, :tec_value])
    CSV.write("test.csv",t, append=true)
    start += map_tec_values
    stop += map_tec_values
end


Start 1
Start 5184
Start 10367
Start 15550
Start 20733
Start 25916
Start 31099
Start 36282
Start 41465
Start 46648
Start 51831
Start 57014
Start 62197